In [3]:
import networkx as nx
import pickle
import random
import scipy
from scipy.sparse import tril, find
import django
import sys, os
import pandas as pd
import matplotlib.pyplot as plt
import igraph
from django.core.management import call_command

sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from scoping.models import *
from cities.models import *


In [9]:
q = Query.objects.get(pk=7965)

pd.set_option('display.max_colwidth', -1)
dis = []
pat = "[a-z|A-Z]{2} [0-9]{5}"
for d in q.doc_set.all():
    if d.docauthinst_set.exists():
        fai = d.docauthinst_set.order_by('position').first().institution
        if fai=="" or fai==[""] or fai=="['']":
            continue
        cstring = fai.split(',')[-1]
        country = None
        cstring = cstring.replace('.','')
        if "USA" in cstring:
            country = Country.objects.get(name="United States")
        if country is None:
            if "Trinid & Tobago" in cstring:
                country = Country.objects.get(name="Trinidad and Tobago")
        if country is None:
            for ukc in ["england","scotland","wales","northern Ireland"]:
                if ukc in cstring.lower():
                    country = Country.objects.get(name="United Kingdom") 
                    
        if country is None:
            if "DEM REP CONGO" in cstring:
                country = Country.objects.get(name__icontains="Democratic Republic of the Congo")
            if "Cote Ivoire" in cstring:
                country = Country.objects.get(name__icontains="Ivory Coast")
            if "Papua New".lower() in cstring.lower():
                country = Country.objects.get(name="Papua New Guinea")
            if "Fr Polynesia".lower() in cstring.lower():
                country = Country.objects.get(name="French Polynesia")
            if "Byelarus".lower() in cstring.lower():
                country = Country.objects.get(name="Belarus")
            if "Rom~nia" in cstring:
                country = Country.objects.get(name="Romania")
            if "Neth Antilles" in cstring:
                country = Country.objects.get(name="Netherlands Antilles")
            if "St Kitts & Nevi".lower() in cstring.lower():
                country = Country.objects.get(name="Saint Kitts and Nevis")
            if "Saint Vincent".lower() in cstring.lower():
                country = Country.objects.get(name="Saint Vincent and the Grenadines")
            if "Cent Afr Republ".lower() in cstring.lower():
                country = Country.objects.get(name="Central African Republic")
            if "Marshall Isl".lower() in cstring.lower():
                country = Country.objects.get(name="Marshall Islands")
            if "Falkland Isl".lower() in cstring.lower():
                country = Country.objects.get(name="Falkland Islands")
            if "Turks and Caicos".lower() in cstring.lower() or "Turks & Caicos".lower() in cstring.lower():
                country = Country.objects.get(name="Turks and Caicos Islands")
            if "Barbados".lower() in fai.lower(): # the last part is sometimes W Ind..
                country = Country.objects.get(name="Barbados")
            if "Antigua".lower() in cstring.lower():
                country = Country.objects.get(name="Antigua and Barbuda")
            if "St Vincent".lower() in cstring.lower():
                country = Country.objects.get(name="Saint Vincent and the Grenadines")
                
        if country is None:
            if "Mongol Peo Rep".lower() in cstring.lower():
                country = Country.objects.get(name="Mongolia")

        if country is None:
            for c in Country.objects.all():
                if c.name.lower() in cstring.lower() or c.name.lower() in cstring.replace('U ','United ').lower():
                    country = c
                    break
        if country is None:
            for c in Country.objects.all():
                for alt in c.alt_names.all():
                    if alt.name.lower() in cstring.lower():
                        country = c
                        break
                        
                        
        if country is None:
            if re.match(pat, cstring):
                country = Country.objects.get(name="United States")
            if "washington" in fai.lower() or "COLLINS,CO".lower() in fai.lower() or "CHAMPAIGN,IL".lower()  in fai.lower() or "OAKLAND,CA".lower() in fai.lower():
                country = Country.objects.get(name="United States")
            
        if country is None:
            print(cstring, fai)
            continue
                
        dis.append({"id":d.id, "fai": fai,"country": country.name, "continent": country.continent.name})
    else:
        pass #print(d.title)
df = pd.DataFrame.from_dict(dis)

df.head()

 * Univ Virginia, Dept Civil & Environm Engn, *, *, *.


,continent,country,fai,id
0,Europe,United Kingdom,"Liverpool John Moores Univ, Res Ctr Evolutionary Anthropol & Palaeoecol, Sch Nat Sci & Psychol, Liverpool L3 3AF, Merseyside, England.",1095
1,Europe,Austria,"Leopold Franzens Univ Innsbruck, Inst Bot, A-6020 Innsbruck, Austria.",1133
2,North America,United States,"Arizona State Univ, Sch Sustainabil, Tempe, AZ 85287 USA.",1135
3,North America,United States,"Columbia Univ, Dept Econ, 1022 Int Affairs Bldg,MC 3308,420 West 118th St, New York, NY 10027 USA.",1152
4,Europe,Poland,"Panstwowy Inst Badawczy, Inst Ochrony Srodowiska, Warsaw, Poland.",1164


In [11]:
df.to_csv('data/affiliation_countries.csv',index=False)

In [12]:
df.shape

(308309, 4)